Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [2]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))
# YOUR CODE HERE
df = df.set_index('id')

df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


**Test 2**

In [3]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [4]:
target = 'status_group'

# YOUR CODE HERE

y = df[target]
X = df.drop(target, axis=1)

**Test 3**

In [5]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [6]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_val = train_test_split(X, train_size=0.80, test_size=0.20,
                             stratify=df['status_group'])

y_train, y_val = train_test_split(y, train_size=0.80, test_size=0.20,
                             stratify=df['status_group'])

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 39), (11880, 39), (47520,), (11880,))

**Test 4**

In [7]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [8]:
# YOUR CODE HERE
baseline_acc = y_train.value_counts(normalize=True)[0]

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5430765993265994


**Task 5**

In [9]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [10]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce

# YOUR CODE HERE
import numpy as np

def wrangle(X):
    X = X.copy()
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    cols_with_zeros = ['longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    X = X.drop(columns='quantity_group')
    return X

In [11]:
X_train = wrangle(X_train)
X_val = wrangle(X_val)

In [12]:
X_train.value_counts().to_frame()

0
amount_tsh date_recorded funder                 gps_height installer        longitude latitude   wpt_name             num_private basin                   subvillage region      region_code district_code lga              ward         population public_meeting recorded_by             scheme_management scheme_name            permit construction_year extraction_type extraction_type_group extraction_type_class management management_group payment        payment_type water_quality quality_group quantity     source source_type source_class waterpoint_type             waterpoint_type_group   
0.0        2004-03-01    Caltus                 1856       Cultus           31.539761 -7.983106  Kwa Pambajila        0           Lake Tanganyika         Magomeni   Rukwa       15          4             Sumbawanga Urban Kasense      900        True           GeoData Consultants Ltd VWC               Mradi wa maji wa kasen True   2005              gravity         gravity               gravity               vwc        user-group       never pay      never pay    soft          good          dry          spring spring      groundwater  communal standpipe          communal standpipe     1
30.0       2013-01-18    Government Of Tanzania 155        District Council 39.289690 -10.315892 Kwa Paul Nyagaya     0           Ruvuma / Southern Coast Nakadi     Lindi       80          23            Lindi Rural      Nyangao      1          False          GeoData Consultants Ltd WUA               Nyangao Water Supply   False  2010              gravity         gravity               gravity               wua        user-group       pay per bucket per bucket   soft          good          enough       river  river/lake  surface      communal standpipe multiple communal standpipe     1
           2013-01-19    Government Of Tanzania 83         District Council 39.376216 -10.294355 Kwa Mtanga           0           Ruvuma / Southern Coast Mkunguni A Lindi       80          23            Lindi Rural      Mtama        1          False          GeoData Consultants Ltd WUA               Nyangao Water Supply   False  2010              gravity         gravity               gravity               wua        user-group       pay per bucket per bucket   soft          good          insufficient river  river/lake  surface      communal standpipe multiple communal standpipe     1
                                                81         District Council 39.369811 -10.302435 Kwa Mzee Kalikonya   0           Ruvuma / Southern Coast Mihogoni   Lindi       80          23            Lindi Rural      Mtama        1          False          GeoData Consultants Ltd WUA               Nyangao Water Supply   False  2010              gravity         gravity               gravity               wua        user-group       pay per bucket per bucket   soft          good          insufficient river  river/lake  surface      communal standpipe multiple communal standpipe     1
                                                77         District Council 39.380568 -10.291267 Kwa Ntenje           0           Ruvuma / Southern Coast Namindondi Lindi       80          23            Lindi Rural      Mtama        1          False          GeoData Consultants Ltd WUA               Nyangao Water Supply   False  2010              gravity         gravity               gravity               wua        user-group       pay per bucket per bucket   soft          good          enough       river  river/lake  surface      communal standpipe multiple communal standpipe     1
...                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [13]:
numeric_features = X_train.select_dtypes(include='number').columns.tolist()

cardinality = X_train.select_dtypes(exclude='number').nunique()

categorical_features = cardinality[cardinality <= 50].index.tolist()

'''
features = numeric_features + categorical_features , 
            'region_code', 'management', 'construction_year', 'basin', 
            'quality_group', 'waterpoint_type', 'management_group', 
            'payment_type', 'population'
'''
features = ['gps_height', 'latitude', 'longitude']
X_train = X_train[features]
X_val = X_val[features]

In [14]:
for depth in [1]:
    model = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),
        SimpleImputer(strategy='mean'),
        DecisionTreeClassifier(min_samples_leaf=20, max_depth=depth, random_state=42)
    )

    model.fit(X_train, y_train);
    
    training_acc = model.score(X_train, y_train)
    validation_acc = model.score(X_val, y_val)

    print('Train Accuracy', depth,'|', model.score(X_train, y_train))
    print('Validation Accuracy', depth, '|', model.score(X_val, y_val))

Train Accuracy 1 | 0.5430765993265994
Validation Accuracy 1 | 0.5430976430976431


C:\Users\benja\anaconda3\envs\unit2\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [16]:
'''model = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),
        SimpleImputer(strategy='mean'),
        DecisionTreeClassifier(min_samples_leaf=20, random_state=42)
    )

model.fit(X_train, y_train);

training_acc = model.score(X_train, y_train)
validation_acc = model.score(X_val, y_val)
'''

"model = make_pipeline(\n        ce.OneHotEncoder(use_cat_names=True),\n        SimpleImputer(strategy='mean'),\n        DecisionTreeClassifier(min_samples_leaf=20, random_state=42)\n    )\n\nmodel.fit(X_train, y_train);\n\ntraining_acc = model.score(X_train, y_train)\nvalidation_acc = model.score(X_val, y_val)\n"

**Test 6**

In [17]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [18]:
# YOUR CODE HERE
for depth in [1]:
    model = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),
        SimpleImputer(strategy='mean'),
        DecisionTreeClassifier(min_samples_leaf=20, max_depth=depth, random_state=42)
    )

    model.fit(X_train, y_train);
    
    training_acc = model.score(X_train, y_train)
    validation_acc = model.score(X_val, y_val)
    
print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.5430765993265994
Validation Accuracy: 0.5430976430976431


C:\Users\benja\anaconda3\envs\unit2\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


**Test 7**

In [19]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

In [ ]:
X_train.dtypes

In [ ]:
tree = DecisionTreeClassifier().fit(X_train, y_train)
tree.feature_importances_

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [ ]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
# YOUR CODE HERE
raise NotImplementedError()

submission.head()

**Test 8**

In [ ]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
raise NotImplementedError()